In [18]:
from app.dotenv import base_dir, data_dir
from app.model.model import  Model
import os

In [19]:
print('dotenv params:', {os.getenv('PARAM1')})
print('base_dir:', base_dir)
print('data_dir:', data_dir)

model = Model()
print('model:', model)

dotenv params: {'PARAM1_value'}
base_dir: c:\Users\yili0901\Downloads\code\ivrit_ai\app
data_dir: c:\Users\yili0901\Downloads\code\ivrit_ai\data
model: loaded Model


In [20]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset

# ds = load_dataset("ivrit-ai/audio-transcripts")
dataset = load_dataset('C:/Users/yili0901/Downloads/code/ivrit_ai/data/audio-transcripts',split="train")

In [21]:
print(dataset)
print(dataset['uuid'][0])  # Print the first example in the training set

Dataset({
    features: ['source', 'episode', 'uuid', 'text', 'attrs'],
    num_rows: 2183042
})
AlHalal/2020.05.04 פרק 9 - די-מארס/0
